In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch import functional as F
from torch.autograd import Variable
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Натренируем модель

In [3]:
batch_size = 64

In [4]:
train_loader = DataLoader(MNIST('.', train=True, download=True,
                                transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])), 
                          batch_size=batch_size, shuffle=True)

test_loader = DataLoader(MNIST('.', train=False, download=True,
                               transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))])),
                         batch_size=batch_size, shuffle=True)

In [5]:
model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(784, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
        ).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [6]:
max_epochs = 1
for epoch in range(max_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if not batch_idx % 100:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({(100. * batch_idx / len(train_loader)):.0f}%)]\tLoss: {loss.item():.6f}')

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.278261
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.217392
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.311982
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.358940
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.233295
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.158603
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.246624
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.040829
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.239288
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.095774


## Проверим качество

In [7]:
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
print(f'Accuracy: {correct}/{len(test_loader.dataset)} ({(100. * correct / len(test_loader.dataset)):.0f}%)\n')

Accuracy: 9637/10000 (96%)



## Сделаем атаку

In [8]:
def fast_adversarial_example(model, dig, eta=0.1, iters=10):
    x = torch.normal(mean=0.0, std=0.01, size=(1, 1, 28, 28)).to(device)
    x.requires_grad = True
    y = torch.from_numpy(np.array([dig], dtype=np.int64)).to(device)
    for _ in range(iters):
        output = model(x)
        loss = criterion(output, y)
        model.zero_grad()
        if x.grad is not None:
            x.grad.data.fill_(0)
        loss.backward()
        x = torch.clamp(x - eta * x.grad.data.sign(), 0.0, 1.0)
        x = Variable(x.data, requires_grad=True)
    return x

In [9]:
example = fast_adversarial_example(model, 3)

In [10]:
correct = 0
y_true, y_pred = [], []
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        data = torch.max(data * 0.5, 0.5 * example)
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        y_true.extend(list(target.cpu().numpy().reshape(-1)))
        y_pred.extend(list(pred.cpu().numpy().reshape(-1)))
        correct += pred.eq(target.view_as(pred)).sum().item()
print(f'Accuracy: {correct}/{len(test_loader.dataset)} ({(100. * correct / len(test_loader.dataset)):.0f}%)\n')

Accuracy: 1707/10000 (17%)



## Посмотрим на confusion matrix

In [11]:
pd.DataFrame(confusion_matrix(y_true, y_pred), columns=(0,1,2,3,4,5,6,7,8,9))

,0,1,2,3,4,5,6,7,8,9
0,345,0,0,633,0,0,0,0,1,1
1,0,0,0,1135,0,0,0,0,0,0
2,0,0,134,898,0,0,0,0,0,0
3,0,0,0,1010,0,0,0,0,0,0
4,4,0,0,927,5,0,3,0,2,41
5,2,0,0,885,0,4,1,0,0,0
6,9,0,0,767,0,1,177,0,0,4
7,1,0,0,1027,0,0,0,0,0,0
8,1,0,5,955,0,0,0,0,13,0
9,3,0,0,987,0,0,0,0,0,19
